In [1]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

/Users/jmangia/anaconda2/envs/SwiftNLC/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# define documents
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])

In [3]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1

In [4]:
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)

[[6, 2], [3, 1], [7, 4], [8, 1], [9], [10], [5, 4], [11, 3], [5, 1], [12, 13, 2, 14]]


In [5]:
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[ 6  2  0  0]
 [ 3  1  0  0]
 [ 7  4  0  0]
 [ 8  1  0  0]
 [ 9  0  0  0]
 [10  0  0  0]
 [ 5  4  0  0]
 [11  3  0  0]
 [ 5  1  0  0]
 [12 13  2 14]]


In [6]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [7]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [8]:
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [9]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 100)            1500      
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 401       
Total params: 1,901
Trainable params: 401
Non-trainable params: 1,500
_________________________________________________________________
None


In [12]:
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=1)

Epoch 1/50
10/10 [==============================] - 0s 232us/step - loss: 0.1958 - acc: 1.0000
Epoch 2/50
10/10 [==============================] - 0s 185us/step - loss: 0.1939 - acc: 1.0000
Epoch 3/50
10/10 [==============================] - 0s 209us/step - loss: 0.1921 - acc: 1.0000
Epoch 4/50
10/10 [==============================] - 0s 209us/step - loss: 0.1902 - acc: 1.0000
Epoch 5/50
10/10 [==============================] - 0s 237us/step - loss: 0.1884 - acc: 1.0000
Epoch 6/50
10/10 [==============================] - 0s 231us/step - loss: 0.1866 - acc: 1.0000
Epoch 7/50
10/10 [==============================] - 0s 172us/step - loss: 0.1849 - acc: 1.0000
Epoch 8/50
10/10 [==============================] - 0s 284us/step - loss: 0.1831 - acc: 1.0000
Epoch 9/50
10/10 [==============================] - 0s 215us/step - loss: 0.1814 - acc: 1.0000
Epoch 10/50
10/10 [==============================] - 0s 231us/step - loss: 0.1797 - acc: 1.0000
Epoch 11/50
10/10 [==============================

In [13]:
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 100)            1500      
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 401       
Total params: 1,901
Trainable params: 401
Non-trainable params: 1,500
_________________________________________________________________


In [18]:
# test prediction
test = array([[ 6,  2,  0,  0],
              [ 3,  1,  0,  0],
              [ 7,  4,  0,  0],
              [ 8,  1,  0,  0],
              [ 9,  0,  0,  0],
              [10,  0,  0,  0],
              [ 5,  4,  0,  0],
              [11,  3,  0,  0],
              [ 5,  1,  0,  0],
              [12, 13,  2, 14]])

y_pred = model.predict(test)
print(y_pred)

[[0.83540314]
 [0.897266  ]
 [0.9012979 ]
 [0.90883225]
 [0.83869994]
 [0.11772853]
 [0.10637807]
 [0.16841738]
 [0.14953981]
 [0.0196906 ]]


In [ ]:
# save model
model.save('model.h5')

In [ ]:
# export coreml
import coremltools
coreml_model = coremltools.converters.keras.convert(model)
coreml_model.save('model.mlmodel')

In [ ]:
# dump coreml
coreml_model